In [17]:
import numpy as np
import piquasso as pq

import itertools
import functools

from typing import List


updates = 10

shots = 5

M = d = 5  # modes

eta = learning_rate = 0.3


Q = np.random.rand(d, d)


def calculate_QUBO_explicitely(Q):
    d = len(Q)

    bitstrings = list(map(np.array, list(itertools.product([0, 1], repeat=d))))

    values = []

    for bitstring in bitstrings:
        values.append(bitstring @ Q @ bitstring)

    return min(values), bitstrings[np.argmin(values)]



def map_to_bitstring(sample: np.ndarray, parity: int) -> np.ndarray:
    return list(map(lambda x: (x + parity) % 2, sample))


simulator = pq.PureFockSimulator(d=d)

def get_samples(thetas: List[float]):
    iterator = iter(thetas)

    with pq.Program() as program:
        pq.Q(all) | pq.StateVector((1, ) * d)

        for column in range(d):
            start_index = column % 2
            for element in range(start_index, d - 1, 2):
                pq.Q(element, element + 1) | pq.Beamsplitter(next(iterator))

        pq.Q() | pq.ParticleNumberMeasurement()

    return simulator.execute(program, shots).samples


min_energy = None


thetas = np.random.uniform(0.0, 2 * np.pi, d)

def get_energy(thetas: List[float], mapper) -> float:
    samples = get_samples(thetas)
    bitstrings = list(map(mapper, samples))
    energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
    return sum(energies) / shots


for parity in (0, 1):
    map_to_bistring_with_parity = functools.partial(map_to_bitstring, parity=parity)
    for _ in range(updates):

        samples = get_samples(thetas)

        bitstrings = list(map(map_to_bistring_with_parity, samples))
        energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
        energy_mean = sum(energies) / shots

        print("Energies:", energies)

        if min_energy is None or min(energies) < min_energy:
            min_energy = min(energies)
            min_bistring = bitstrings[np.argmin(energies)]

        derivatives = []

        for j in range(len(thetas)):
            upshifted_thetas = np.copy(thetas)
            upshifted_thetas[j] += np.pi / 2
            upshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

            downshifted_thetas = np.copy(thetas)
            downshifted_thetas[j] -= np.pi / 2
            downshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

            derivative = (upshifted_energy - downshifted_energy) / 2

            thetas[j] -= eta * derivative

print("Algo:", min_energy, min_bistring)

print("Exact solution:", calculate_QUBO_explicitely(Q))


StopIteration: 